# causal bounding across multiple domains
---

In [1]:
import cvxpy as cp
import numpy as np
from itertools import product

### configurations

In [2]:
I  = (0, 1)
J  = (0, 1, 2, 3)
X  = (0, 1)
Y  = (0, 1)

In [3]:
FLAG_SELECT_X  = False  # selection node on X?
FLAG_OBSX_DIFF = False  # obs. marginal distribution of X differ across domains?
FLAG_OBS_SAME  = False  # obs. joint distribution differ across domains? (overrides FLAG_OBSX_DIFF)

### observational distribution

In [4]:
# single domain
OBS_DIST   = np.array([[0.4, 0.1],
                       [0.1, 0.4]])

# multiple domains
OBS_DIST_t = np.array([[0.4, 0.1],
                       [0.1, 0.4]])
if FLAG_OBSX_DIFF:
    OBS_DIST_s = np.array([[0.3, 0.1],
                           [0.2, 0.4]])
    src = 's+'
else:
    OBS_DIST_s = np.array([[0.3, 0.2],
                           [0.1, 0.4]])
    src = 's++'
if FLAG_OBS_SAME:
    OBS_DIST_s = OBS_DIST_t
    src = 's'

In [5]:
def P(x=None, y=None, dist=OBS_DIST):
    for v in (x, y):
        if v not in (None, 0, 1):
            return 0.
    x = x if x is not None else slice(None)
    y = y if y is not None else slice(None)
    return dist[x, y].sum()

In [6]:
print(f'target domain: P(X=0) = {P(x=0, dist=OBS_DIST_t):.3f}, P(Y=0) = {P(y=0, dist=OBS_DIST_t):.3f}')
print(f'source domain: P(X=0) = {P(x=0, dist=OBS_DIST_s):.3f}, P(Y=0) = {P(y=0, dist=OBS_DIST_s):.3f}')

target domain: P(X=0) = 0.500, P(Y=0) = 0.500
source domain: P(X=0) = 0.500, P(Y=0) = 0.400


### bounding within one domain

In [7]:
q = list()
for i in I:
    qi = list()
    for j in J:
        qi.append(cp.Variable(name='q_{}{}'.format(i,j)))
    q.append(qi)

In [8]:
# simplex constraints
constraints = [q[i][j] >= 0 for i, j in product(I, J)]
s = 0
for i, j in product(I, J):
    s += q[i][j]
constraints += [s == 1]

In [9]:
# observational constraints
constraints += [
    q[0][0] + q[0][1] == OBS_DIST_t[0][0],
    q[0][2] + q[0][3] == OBS_DIST_t[0][1],
    q[1][0] + q[1][2] == OBS_DIST_t[1][0],
    q[1][1] + q[1][3] == OBS_DIST_t[1][1],
    ]

In [10]:
objectives = [
    [q[0][0] + q[1][0] + q[0][1] + q[1][1],
     q[0][2] + q[1][2] + q[0][3] + q[1][3],],
    [q[0][0] + q[1][0] + q[0][2] + q[1][2],
     q[0][1] + q[1][1] + q[0][3] + q[1][3],]
    ]

In [11]:
for x, y in product(X, Y):
    mi = cp.Problem(cp.Minimize(objectives[x][y]), constraints).solve()
    ma = cp.Problem(cp.Maximize(objectives[x][y]), constraints).solve()
    print(f'{mi:.3f} <= P(Y={y}|do(X={x})) <= {ma:.3f}')

0.400 <= P(Y=0|do(X=0)) <= 0.900
0.100 <= P(Y=1|do(X=0)) <= 0.600
0.100 <= P(Y=0|do(X=1)) <= 0.600
0.400 <= P(Y=1|do(X=1)) <= 0.900


### bounding across multiple domains

In [12]:
q = list()
for i in I:
    qi = list()
    for j in J:
        qi.append(cp.Variable(name='q_{}{}'.format(i,j)))
    q.append(qi)

w = list()
for i in I:
    wi = list()
    for j in J:
        wi.append(cp.Variable(name='w_{}{}'.format(i,j)))
    w.append(wi)

In [13]:
# simplex constraints

constraints = list()

constraints += [q[i][j] >= 0 for i, j in product(I, J)]
s = 0
for i, j in product(I, J):
    s += q[i][j]
constraints += [s == 1]

constraints += [w[i][j] >= 0 for i, j in product(I, J)]
s = 0
for i, j in product(I, J):
    s += w[i][j]
constraints += [s == 1]

In [14]:
# observational constraints

constraints += [
    q[0][0] + q[0][1] == OBS_DIST_t[0][0],
    q[0][2] + q[0][3] == OBS_DIST_t[0][1],
    q[1][0] + q[1][2] == OBS_DIST_t[1][0],
    q[1][1] + q[1][3] == OBS_DIST_t[1][1],
    ]

constraints += [
    w[0][0] + w[0][1] == OBS_DIST_s[0][0],
    w[0][2] + w[0][3] == OBS_DIST_s[0][1],
    w[1][0] + w[1][2] == OBS_DIST_s[1][0],
    w[1][1] + w[1][3] == OBS_DIST_s[1][1],
    ]

In [15]:
# cross-domain constraints

if FLAG_SELECT_X:
    # selection node on X (f_X variant, f_Y invariant)
    for j in J:
        sq = 0
        sw = 0
        for i in I:
            sq += q[i][j]
            sw += w[i][j]
        constraints += [sq == sw]
else:
    # selection node on Y (f_X invariant, f_Y variant)
    for i in I:
        sq = 0
        sw = 0
        for j in J:
            sq += q[i][j]
            sw += w[i][j]
        constraints += [sq == sw]

In [16]:
objectives = [
    [q[0][0] + q[1][0] + q[0][1] + q[1][1],
     q[0][2] + q[1][2] + q[0][3] + q[1][3],],
    [q[0][0] + q[1][0] + q[0][2] + q[1][2],
     q[0][1] + q[1][1] + q[0][3] + q[1][3],]
    ]

In [17]:
print(f'source domain: {src}')
for x, y in product(X, Y):
    mi = cp.Problem(cp.Minimize(objectives[x][y]), constraints).solve()
    ma = cp.Problem(cp.Maximize(objectives[x][y]), constraints).solve()
    print(f'{mi:.2f} <= P(Y={y}|do(X={x})) <= {ma:.2f}')

source domain: s++
0.40 <= P(Y=0|do(X=0)) <= 0.90
0.10 <= P(Y=1|do(X=0)) <= 0.60
0.10 <= P(Y=0|do(X=1)) <= 0.60
0.40 <= P(Y=1|do(X=1)) <= 0.90
